# NITROGEN ACCOUNTANCY IN SPACE AGRICULTURE
## Plotting MEC output
---

## Crop Calculations

In [ ]:
from DU4 import *
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import scipy.integrate as integrate
import functools as ft
from matplotlib.pyplot import cm
import pickle

# Make plots look good according to KY
import matplotlib.style as style
plt.rcParams.update(plt.rcParamsDefault)
# style.use('seaborn-poster')
plt.rcParams['figure.dpi'] = 300

'''
Fonts
'''
plt.rcParams['font.family'] = "TeX Gyre Termes"

'''
LaTeX
'''
plt.rcParams['text.usetex'] = True
plt.rcParams['text.latex.preamble'] = '\n'.join([
    r'\usepackage[T3,T1]{fontenc}',
    r'\DeclareSymbolFont{tipa}{T3}{cmr}{m}{n}',
    r'\DeclareMathAccent{\invbreve}{\mathalpha}{tipa}{16}',
    r'\usepackage{siunitx}'
])  # Preamble must be one line!

img_out_path = "./MEC-images/"
data_out_path = "./MEC-mB/"
pickle_path = "./pickles/"

In [ ]:
directory = 'parameter-lists/'
filename = 'crop_parameters_FPSD.xlsx'

filename_full = directory + filename
df = pd.read_excel(filename_full, sheet_name='PxC')
crop_types = df['crop_type']

In [ ]:
Phi_gamma = 500
c_CO2 = 1200
mB_color = '#0066ff' # '#2381A9' # '#187b3f' '#427af0'
dmBdt_color = '#ff6633' # '#471323' #'#2381A9' #'#540025' '#f0b842'

In [ ]:
for crop_type in [crop_types[1]]:
    crop = Crop(crop_type, filename_full=filename_full)

    t = np.arange(0,crop.t_M)

    crop.Y_Qmax = calc_Y_Qmax(crop, Phi_gamma, c_CO2)
    crop.Phi_gammaE = calc_Phi_gammaE(crop, Phi_gamma)
    crop.t_A = calc_t_A(crop, c_CO2, Phi_gamma)

    Y_Q=[]
    eta_C=[]
    A=[]
    n_C=[]
    n_O2=[]
    m_B=[]
    V_trs=[]
    
    for i in t:
        Y_Q.append(calc_Y_Q(i, crop, c_CO2, Phi_gamma))
        eta_C.append(calc_eta_C(i, crop))
        A.append(calc_A(i, crop, c_CO2, Phi_gamma))
        n_C.append(calc_n_C(i, crop, c_CO2, Phi_gamma))
        n_O2.append(calc_n_O2(i, crop, c_CO2, Phi_gamma))
        m_B.append(calc_m_B(i, crop, c_CO2, Phi_gamma))
        V_trs.append(calc_V_trs(i, crop, c_CO2, Phi_gamma))
    
    m_T = np.cumsum(m_B)
    m_E = m_T * crop.f_E

    stuff = [eta_C, Y_Q, A, n_C, n_O2, m_B, m_T, V_trs]
    
    names = [r'$\eta_\text{C}$', r'$Y_\text{Q}$', r'$A$', r'$\invbreve{\dot{n}}_\text{C}$', r'$\invbreve{\dot{n}}_\text{O2}$', r'$\invbreve{\dot{m}}_B$', r'$\invbreve{m}_\text{E}$', r'$\invbreve{\dot{V}}_\text{trs}$']
    ylabels = [r'[frac]', r'$[\si{\mole\of{C,fixed}\per\mole\of{\gamma,absorbed}}]$', r'[frac]', r'[$\si{\mole\of{C}\per\day\per\meter\squared}$]', r'$[\si{\mole\of{O2}\per\day\per\meter\squared}]$', r'$[\si{\gram\per\day\per\meter\squared}]$', r'$[\si{\gram\per\meter\squared}]$', r'$[\si{\liter\per\day\per\meter\squared}]$']
#     fig = plt.figure()
    fig = plt.figure(figsize=[5,10])
    for i in range(0,len(stuff)):
        ax = fig.add_subplot(4,2,i+1)
        ax.plot(t, stuff[i], label = names[i])
        ax.set_xlabel(r'Time, t $[\si{\day\of{AE}}]$')
        ax.set_ylabel(ylabels[i])
        ax.set_title(names[i])
    fig.suptitle(str(crop.Crop) + ', ' + '$\Phi_\gamma=$' + str(Phi_gamma) + r' $\si{\micro\mole\of{\gamma}\per\meter\squared\per\second}$, ' + r'$c_\text{CO2}=$' + str(c_CO2) + r" $\si{\micro\mole\of{CO2}\per\mole\of{air}}$", fontsize=20)
    fig.tight_layout()
    fig.subplots_adjust(top=0.9)
    fig.savefig(img_out_path + str(crop.Crop)+'-MEC-parameters.png', dpi=300, bbox_inches='tight', transparent=True)

plt.show()

In [ ]:
df

In [ ]:
## Plot Basics for Single Crop

# fig, ax1 = plt.subplots()
# ax1.axvline(x=crop.T_a, color='k', linewidth=3, linestyle='-')
# ax1.axvline(x=crop.T_e, color='k', linewidth=3, linestyle='--')
# ax1.axvline(x=crop.T_m, color='k', linewidth=3, linestyle='-.')
# ax1.axvline(x=crop.T_q, color='k', linewidth=3, linestyle=':')
# ax1.set_ylabel('Total Crop Biomass (TCB) [g/m$^{2}$]', fontsize=15)
# ax1.set_xlabel('Time [d$_{AE}$]', fontsize=15)
# ax1.plot(t,TCB,linewidth=5,color='#427af0')
# # f_TCB = interp1d(t,TCB)

# ax2 = ax1.twinx()
# ax2.plot(t,CGR,linewidth=5,color='#f0b842')
# ax2.set_ylabel('Crop Growth Rate (CGR) [g/m$^{2}\cdot d$]', fontsize=15)

fig, ax1 = plt.subplots(figsize=(5,4))
ax1.axvline(x=crop.t_A, color='k', linewidth=3, linestyle='-', label=r"$t_\text{A}$")
ax1.axvline(x=crop.t_E, color='k', linewidth=3, linestyle='--', label=r"$t_\text{E}$")
ax1.axvline(x=crop.t_M, color='k', linewidth=3, linestyle='-.', label=r"$t_\text{M}$")
ax1.axvline(x=crop.t_Q, color='k', linewidth=3, linestyle=':')
ax1.set_ylabel("$\invbreve{m}_\mathrm{T}\ [\si{\gram\of{DW}\per\meter\squared}]$", fontsize=20, color=mB_color)
ax1.set_xlabel("Time, $t\ [\si{\day\of{AE}}]$", fontsize=20)
ax1.plot(t,m_T,linewidth=5,color=mB_color)
# f_TCB = interp1d(t,TCB)

ax2 = ax1.twinx()
ax2.plot(t,m_B,linewidth=5,color=dmBdt_color)
ax2.set_ylabel("$\invbreve{\dot{m}}_\mathrm{B}\ [\si{\gram\of{DW}\per\day\per\meter\squared}]$", fontsize=20,color=dmBdt_color)
ax2.set_title(str(crop.Crop) + ' Biomass Accumulation', fontsize=20)

for label in (ax1.get_xticklabels() + ax1.get_yticklabels() + ax2.get_xticklabels() + ax2.get_yticklabels()):
    label.set_fontsize(14)
    
ax1.legend()
fig.savefig(img_out_path + str(crop.Crop) + ' Biomass Accumulation.png', bbox_inches='tight', dpi=300, transparent=True)
plt.show()

## Total Mass and Change in Mass individial

In [ ]:
%%time
# f = ft.partial(eo.growth_integrand2, crop)
# y1 = f(t)
# integral = integrate.cumtrapz(y1,t)
# plt.plot(integral)

i = 0

for crop_type in crop_types:
    crop = Crop(crop_type, filename_full=filename_full)

    t = np.arange(0,crop.t_M)

    crop.Y_Qmax = calc_Y_Qmax(crop, Phi_gamma, c_CO2)
    crop.Phi_gammaE = calc_Phi_gammaE(crop, Phi_gamma)
    crop.t_A = calc_t_A(crop, c_CO2, Phi_gamma)

    Y_Q=[]
    eta_C=[]
    A=[]
    n_C=[]
    n_O2=[]
    m_B=[]
    V_trs=[]
    
    for j in t:
        Y_Q.append(calc_Y_Q(j, crop, c_CO2, Phi_gamma))
        eta_C.append(calc_eta_C(j, crop))
        A.append(calc_A(j, crop, c_CO2, Phi_gamma))
        n_C.append(calc_n_C(j, crop, c_CO2, Phi_gamma))
        n_O2.append(calc_n_O2(j, crop, c_CO2, Phi_gamma))
        m_B.append(calc_m_B(j, crop, c_CO2, Phi_gamma))
        V_trs.append(calc_V_trs(j, crop, c_CO2, Phi_gamma))

    m_T = integrate.cumtrapz(m_B,x=t,initial=0)
    
    # dump to csv
    arr = np.column_stack((t, m_T))
    np.savetxt(data_out_path + crop.Crop + " MEC mB.csv", arr, delimiter=",")
        
    
    # m_E = m_T * crop.f_E
    
    fig, ax1 = plt.subplots(figsize=(5,4))
    ax1.axvline(x=crop.t_A, color='k', linewidth=3, linestyle='-', label=r"$t_\mathrm{A}$")
    ax1.axvline(x=crop.t_E, color='k', linewidth=3, linestyle='--', label=r"$t_\mathrm{A}$")
    ax1.axvline(x=crop.t_M, color='k', linewidth=3, linestyle='-.', label=r"$t_\mathrm{A}$")
    ax1.axvline(x=crop.t_Q, color='k', linewidth=3, linestyle=':', label=r"$t_\mathrm{Q}$")
    ax1.set_ylabel('$\invbreve{m}_\mathrm{T}\ [\si{\gram\of{DW}\per\meter\squared}]$', fontsize=15, color=mB_color)
    ax1.set_xlabel('Time, $t\ [\si{\day\of{AE}}]$', fontsize=15)
    ax1.plot(t,m_T,linewidth=5,color=mB_color)

    ax2 = ax1.twinx()
    ax2.plot(t,m_B,linewidth=5,color=dmBdt_color)
    ax2.set_ylabel('$\invbreve{\dot{m}}_\mathrm{B}\ [\si{\gram\of{DW}\per\day\per\meter\squared}]$', fontsize=15,color=dmBdt_color)
    ax2.set_title(str(df['Crop'][i]) + ' Biomass Accumulation', fontsize=20)

    for label in (ax1.get_xticklabels() + ax1.get_yticklabels() + ax2.get_xticklabels() + ax2.get_yticklabels()):
        label.set_fontsize(14)
    fig.savefig(img_out_path + 'Crop Biomass Accumulation ' + str(df['Crop'][i]), bbox_inches='tight', dpi=300)
    
    i = i + 1
    
plt.show()

## Total mass and change in mass combined

In [ ]:
%%time

time_fig, time_axs = plt.subplots(3,3, figsize=(10,8))
time_axes = time_axs.ravel()

i = 0

for crop_type in crop_types:
    crop = Crop(crop_type, filename_full=filename_full)

    t = np.arange(0,crop.t_M)

    crop.Y_Qmax = calc_Y_Qmax(crop, Phi_gamma, c_CO2)
    crop.Phi_gammaE = calc_Phi_gammaE(crop, Phi_gamma)
    crop.t_A = calc_t_A(crop, c_CO2, Phi_gamma)

    Y_Q=[]
    eta_C=[]
    A=[]
    n_C=[]
    n_O2=[]
    m_B=[]
    V_trs=[]
    
    for j in t:
        Y_Q.append(calc_Y_Q(j, crop, c_CO2, Phi_gamma))
        eta_C.append(calc_eta_C(j, crop))
        A.append(calc_A(j, crop, c_CO2, Phi_gamma))
        n_C.append(calc_n_C(j, crop, c_CO2, Phi_gamma))
        n_O2.append(calc_n_O2(j, crop, c_CO2, Phi_gamma))
        m_B.append(calc_m_B(j, crop, c_CO2, Phi_gamma))
        V_trs.append(calc_V_trs(j, crop, c_CO2, Phi_gamma))

    m_T = integrate.cumtrapz(m_B,x=t,initial=0)
    
    # m_E = m_T * crop.f_E
     
    time_axes[i].axvline(x=crop.t_A, color='k', linewidth=2, linestyle='-')
    time_axes[i].axvline(x=crop.t_E, color='k', linewidth=2, linestyle='--')
    time_axes[i].axvline(x=crop.t_M, color='k', linewidth=2, linestyle='-.')
    time_axes[i].axvline(x=crop.t_Q, color='k', linewidth=2, linestyle=':')

    time_axes[i].tick_params(axis='y', which='major', colors=mB_color)
    #We are plotting TOTAL biomass, not edible. Edible goes on the PPF vs. CO2 contour plots.
    time_axes[i].plot(t,m_T,linewidth=3,color=mB_color)
    time_axes[i].locator_params(min_n_ticks=4, nbins=4)
    
    time_ax2 = time_axes[i].twinx()
    time_ax2.tick_params(axis='y', which='major', colors=dmBdt_color)
    time_ax2.plot(t,m_B,linewidth=3,color=dmBdt_color)
    time_ax2.locator_params(nbins=4) 
    
    if i == 3:
        time_axes[3].set_ylabel('$\invbreve{m}_\mathrm{T}\ [\si{\gram\of{DW}\per\meter\squared}]$', fontsize=24, color=mB_color)
    elif i == 5:
        time_ax2.set_ylabel('$\invbreve{\dot{m}}_\mathrm{B}\ [\si{\gram\of{DW}\per\day\per\meter\squared}]$', fontsize=24, color=dmBdt_color)
    
#     time_ax2.set_title(str(df['Crop'][i]), fontsize=20)

    # This handles all label sizes
    for label in (time_axes[i].get_xticklabels() + time_axes[i].get_yticklabels() + time_ax2.get_xticklabels() + time_ax2.get_yticklabels()):
        label.set_fontsize(15)
        
    i = i + 1

time_fig.suptitle('Total Biomass and Crop Growth Rate', fontsize=24, y=1)
time_axes[7].set_xlabel('$t\ [\si{\day\of{AE}}]$', fontsize=24)
time_fig.legend(labels=[r"$t_\mathrm{A}$",r"$t_\mathrm{E}$",r"$t_\mathrm{M}$","$t_\mathrm{Q}$"], ncol=4, loc="lower center",  bbox_to_anchor=(0.5, -0.075), fontsize=15)
time_fig.tight_layout()
plt.show()

time_fig.savefig(img_out_path + 'Total Biomass and Crop Growth Rate.png', bbox_inches='tight', dpi=300)

## Plot Countours from pickled data

In [ ]:
con_fig, con_axs = plt.subplots(3,3, figsize=(10,8))
con_axes = con_axs.ravel()

i=0

for crop_type in crop_types:
    data_pickle = pickle.load(open(pickle_path + str(crop_types[i]) + "_pickle.pkl", 'rb'))
    contour = con_axes[i].contourf(data_pickle['X'],data_pickle['Y'],data_pickle['m_E'],cmap='YlGn', color='w')
#     con_axes[i].text(.5,0.1,str(df['Crop'][i]), horizontalalignment='center',transform=con_axes[i].transAxes,fontsize=16,color='k',bbox=dict(facecolor='w', alpha=0.4))
    for label in (con_axes[i].get_xticklabels() + con_axes[i].get_yticklabels()):
        label.set_fontsize(15)
    con_axes[i].locator_params(nbins=4)
    i += 1

con_fig.tight_layout()
con_axes[7].set_xlabel(r'$\Phi_\gamma\ [\si{\micro\mole\of{\gamma}\per\meter\squared\per\second}]$', fontsize=24)
con_axes[3].set_ylabel(r'$c_\mathrm{CO2}\ [\si{\micro\mole\of{CO2}\per\mole\of{air}}]$', fontsize=24)
con_fig.suptitle('Edible Biomass at Harvest', fontsize=24, y=1.03)

cb_ax = con_fig.add_axes([1.0,0.05,.05,0.9])
cbar = con_fig.colorbar(contour, ax=con_axes.ravel().tolist(), cax=cb_ax)
cbar.set_label(r'$\invbreve{m}_\mathrm{E}\ @\ t_\mathrm{M}\ [\si{\gram\of{DW}\per\meter\squared}]$', size=24)
cbar.ax.tick_params(labelsize=15) 
con_fig.savefig(img_out_path + 'Edible Biomass at Harvest.png', bbox_inches='tight', dpi=300)

plt.show()

## Plot One Contour

In [ ]:
con_fig, con_axs = plt.subplots(1,1, figsize=(5,4))
con_axes = con_axs
crop_type = "lettuce"

data_pickle = pickle.load(open(pickle_path + str(crop_type) + "_pickle.pkl", 'rb'))
contour = con_axes.contourf(data_pickle['X'],data_pickle['Y'],data_pickle['m_E'],cmap='YlGn', color='w')
for label in (con_axes.get_xticklabels() + con_axes.get_yticklabels()):
    label.set_fontsize(15)
con_axes.locator_params(nbins=4)

# con_fig.tight_layout()
con_axes.set_xlabel(r'$\Phi_\gamma\ [\si{\micro\mole\of{\gamma}\per\meter\squared\per\second}]$', fontsize=24)
con_axes.set_ylabel(r'$c_\mathrm{CO2}\ [\si{\micro\mole\of{CO2}\per\mole\of{air}}]$', fontsize=24)
con_fig.suptitle(str(crop_type).capitalize() + ' Edible Biomass at Harvest', fontsize=24, y=1.03)

cb_ax = con_fig.add_axes([1.0,0.05,.05,0.9])
cbar = con_fig.colorbar(contour, ax=con_axes, cax=cb_ax)
cbar.set_label(r'$\invbreve{m}_\mathrm{E}\ @\ t_\mathrm{M}\ [\si{\gram\of{DW}\per\meter\squared}]$', size=24)
cbar.ax.yaxis.set_tick_params(labelsize=15)


con_fig.savefig(img_out_path + 'Edible Biomass at Harvest ' + str(crop_type) + '.png', bbox_inches='tight', dpi=300)

plt.show()

## Generate and Plot Contours

In [ ]:
# %%time

# n = 5

# def m_E_mesh(Phi_gamma, c_CO2, crop_type):    
#     crop = Crop(crop_type, filename_full=filename_full)
#     crop.Y_Qmax = calc_Y_Qmax(crop, Phi_gamma, c_CO2)
#     crop.Phi_gammaE = calc_Phi_gammaE(crop, Phi_gamma)
#     crop.t_A = calc_t_A(crop, c_CO2, Phi_gamma)
    
#     m_E = calc_m_E(crop,Phi_gamma,c_CO2)

#     return m_E

# m_E_mesh = np.vectorize(m_E_mesh)

# fig, axs = plt.subplots(3,3, figsize=(10,8))
# axes = axs.ravel()
# i = 0

# for crop_type in crop_types:
#     crop = Crop(crop_type, filename_full=filename_full)
#     PPF_vec = np.linspace(crop.PPF_min,crop.PPF_max,n)
#     CO2_vec = np.linspace(crop.CO2_min,crop.CO2_max,n)
#     X,Y = np.meshgrid(PPF_vec,CO2_vec)

#     m_E = m_E_mesh(X,Y,crop_type)
    
#     contour = axes[i].contourf(X,Y,m_E,cmap='plasma', color='k')
#     axes[i].text(.5,.9,str(df['Crop'][i]), horizontalalignment='center',transform=axes[i].transAxes,fontsize=16,color='white')
#     for label in (axes[i].get_xticklabels() + axes[i].get_yticklabels()):
#         label.set_fontsize(14)
    
#     i=i+1

# m_E = m_E_mesh(X,Y,'lettuce')
# #     contour = axes[i].contour(X,Y,TEB, color='k')
#     contour = axes[i].contourf(X,Y,m_E,cmap='plasma', color='k')
#     axes[i].text(.5,.9,str(df['Crop'][i]), horizontalalignment='center',transform=axes[i].transAxes,fontsize=16,color='white')
# #     axes[i].clabel(contour, colors = 'k', fontsize=12, inline=True)
#     for label in (axes[i].get_xticklabels() + axes[i].get_yticklabels()):
#         label.set_fontsize(14)
    
#     i=i+1

# fig.tight_layout()
# axes[7].set_xlabel(r'$\Phi_\gamma\ [\si{\micro\mole\of{\gamma}\per\meter\squared\per\second}]$', fontsize=19)
# axes[3].set_ylabel(r'$c_\text{CO2}\ [\si{\micro\mole\of{CO2}\per\mole\of{air}}]$', fontsize=19)
# fig.suptitle('Edible Biomass at Harvest', fontsize=20, y=1.03)
# cbar = fig.colorbar(contour, ax=axes.ravel().tolist(), shrink=0.95)
# cbar.set_label(r'$\text{Edible Biomass}, \invbreve{m}_\text{E}\ @\ t_\text{M}\ [\si{\gram\of{DW}\per\meter\squared}]$', size=20)
# cbar.ax.tick_params(labelsize=15) 
# fig.savefig('Edible Biomass at Harvest', bbox_inches='tight', dpi=300)


In [ ]:
# fig, ax1 = plt.subplots(figsize=(5,4))
# # ax2 = ax1.twinx()
# n = len(crop_types)
# color=iter(cm.rainbow(np.linspace(0,1,n)))

# for crop_type in crop_types:
#     c=next(color)
#     crop = eo.Crop(crop_type, filename_full=filename_full)

#     t = np.arange(0,100)

#     crop.CQY_max = eo.calc_CQY_max(crop,PPF=PPF, CO2=CO2)
#     crop.PPF_E = eo.calc_PPF_E(crop, PPF=PPF)
#     crop.T_a = eo.calc_T_a(crop, CO2=CO2)
    
#     CQY, CUE, A, DCG, DOP, CGR = eo.calc_growth(t,crop)
# #     TCB = np.cumsum(CGR)
    
# #     ax1.axvline(x=crop.T_a, color=c, linewidth=3, linestyle='-')
# #     ax1.axvline(x=crop.T_e, color=c, linewidth=3, linestyle='--')
# #     ax1.axvline(x=crop.T_m, color=c, linewidth=3, linestyle='-.')
# #     ax1.axvline(x=crop.T_q, color=c, linewidth=3, linestyle=':')
# #     ax1.set_ylabel('B (TCB) [g/m$^{2}$]', fontsize=15, color=c)
#     ax1.set_xlabel('Time [d$_{AE}$]', fontsize=15)
#     ax1.plot(t,CGR,linewidth=5,color=c)
#     ax1.set_xlim(0,100)
    